In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text  # Needed for loading universal-sentence-encoder-cmlm/multilingual-preprocess
import numpy as np

!python -m spacy download en_core_web_sm

2021-07-30 19:19:36.968492: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-30 19:19:36.968647: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/sgizinski/repos/AI-strategies-papers-regulations-monitoring/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


2021-07-30 19:19:40.850176: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-30 19:19:40.850233: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
     |████████████████████████████████| 13.6 MB 1.4 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
def normalization(embeds):
  norms = np.linalg.norm(embeds, 2, axis=1, keepdims=True)
  return embeds/norms


In [3]:
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")


2021-07-30 19:19:56.629476: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-30 19:19:56.629546: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-30 19:19:56.629566: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-8E43FDR): /proc/driver/nvidia/version does not exist
2021-07-30 19:19:56.629977: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-30 19:19:59.383151: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLI

In [4]:
def embedd_sents(sents):
    return normalization(encoder(preprocessor(tf.constant(sents)))["default"])

In [5]:
from mars import db

In [6]:
import spacy

en = spacy.load('en_core_web_sm')

In [7]:
# db.contents.fetch

processed_texts= db.get_collection_or_create(db.db, "ProcessedTexts")
for cont in db.contents.exportDocs():
    text = cont['content']
    sents = [str(s) for s in list(en(text).sents)]
    doc = processed_texts.createDocument()
    doc['textId']=cont['_id']
    doc['sentences']=sents
    doc.save()

# sents = list(en(texts[0]).sents)

# Putting embeddings in DB

In [14]:
for doc in tqdm(processed_texts.fetchAll()):
    doc['sentencesEmbeddings'] = embedd_sents(doc.sentences)
    doc.save()

SyntaxError: invalid syntax (2906820417.py, line 1)

In [ ]:
embedd_sents(doc.sentences)

2021-07-30 19:30:51.132274: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2262171648 exceeds 10% of free system memory.
2021-07-30 19:30:51.286340: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2262171648 exceeds 10% of free system memory.
2021-07-30 19:30:51.481874: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2262171648 exceeds 10% of free system memory.
2021-07-30 19:30:52.946649: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2262171648 exceeds 10% of free system memory.
2021-07-30 19:30:53.470323: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2262171648 exceeds 10% of free system memory.


### Save similarities to DB

In [ ]:
targets = ['privacy protection',
'fairness',
'transparency, openness']
e_tergets = embedd_sents(targets)



for doc in tqdm(processed_texts.fetchAll()):
    search_scores=dict()
    results = np.matmul(e_tergets, np.transpose(e_sents))

    for result, target in zip(results, targets):
        search_scores[target] = list(result)
        
    doc['searchSimilarityScores']=search_scores
    doc.save()

In [6]:
import numpy as np

def get_top_n_indexes(l, n):
    top_n_idx = np.argsort(l)[-n:]
    return top_n_idx

In [4]:

# top_2_values = [a[i] for i in top_2_idx]


In [5]:
top_2_idx

array([0, 2])

### Saving top N sentences

In [38]:
for doc in tqdm(processed_texts.fetchAll()):
    topnsents = dict()
    for target, similarities in doc['searchSimilarityScores'].items():
        idxs = get_top_n_indexes(similarities, 10)
        topn = np.array(doc['sentences'])[idxs]
        topnsents[target] = topn
    doc['topNSents'] = topnsents
    doc.save()

In [64]:
def print_best_sents(e_targets, e_sents, sents, targets):
    r = np.matmul(e_tergets, np.transpose(e_sents))
    for j, (i, target) in enumerate(zip(list(np.argmax(r, axis=1)), targets)):
        print(target)
        print("SCORE:", r[j][i])
        print("BEST HIT:")
        print(sents[i])
        print('-'*30)

In [65]:
print_best_sents(e_tergets, e_sents, sents, targets)

privacy protection
SCORE: 0.28070253
BEST HIT:

    5
    Social and Societal Influences of AI

    AI advances will touch people and society in numerous ways, including potential influences on privacy, democracy, criminal justice, and human rights.
------------------------------
fairness
SCORE: 0.25957346
BEST HIT:
Address such areas as fairness and inclusivity, explanation and transparency, security and privacy, values and ethics, collaboration between people and AI systems, interoperability of systems, and of the trustworthiness, reliability, containment, safety, and robustness of the technology.
------------------------------
transparency, openness
SCORE: 0.32075033
BEST HIT:


    Researchers, officials, and the public should be sensitive to these possibilities and we should seek to develop methods that detect and correct those errors and biases, not replicate them.
------------------------------
